Creating our own MCP Server and MCP Client
The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Use python code made mostly by a hard-working Engineering Team(Creew AI):

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Gulnara")
account

Account(name='gulnara', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [4]:
account.buy_shares("ANET", 3, "Because this company looks promising")

'Completed. Latest details:\n{"name": "gulnara", "balance": 9600.86332, "strategy": "", "holdings": {"ANET": 3}, "transactions": [{"symbol": "ANET", "quantity": 3, "price": 133.04556, "timestamp": "2025-08-20 16:53:05", "rationale": "Because this company looks promising"}], "portfolio_value_time_series": [["2025-08-20 16:53:05", 9999.20332]], "total_portfolio_value": 9999.20332, "total_profit_loss": -0.7966799999994691}'

In [5]:
account.report()

'{"name": "gulnara", "balance": 9600.86332, "strategy": "", "holdings": {"ANET": 3}, "transactions": [{"symbol": "ANET", "quantity": 3, "price": 133.04556, "timestamp": "2025-08-20 16:53:05", "rationale": "Because this company looks promising"}], "portfolio_value_time_series": [["2025-08-20 16:53:05", 9999.20332], ["2025-08-20 16:53:05", 9999.20332]], "total_portfolio_value": 9999.20332, "total_profit_loss": -0.7966799999994691}'

In [6]:
account.list_transactions()

[{'symbol': 'ANET',
  'quantity': 3,
  'price': 133.04556,
  'timestamp': '2025-08-20 16:53:05',
  'rationale': 'Because this company looks promising'}]

Let's write our own MCP server and use it

In [7]:
params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [8]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [9]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Gulnara and my account is under the name Gulnara. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [10]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Gulnara, your current cash balance is approximately $9,600.86. In terms of holdings, you have 3 shares of ANET (Arista Networks). If you need any further details or want to make changes to your account, feel free to ask!

Our own MCP Client

In [11]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [12]:
request = "My name is Gulnara and my account is under the name Gulnara. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your account balance is $9,600.86. If you need any further information or assistance, please let me know!

In [13]:
context = await read_accounts_resource("Gulnara")
print(context)

{"name": "gulnara", "balance": 9600.86332, "strategy": "", "holdings": {"ANET": 3}, "transactions": [{"symbol": "ANET", "quantity": 3, "price": 133.04556, "timestamp": "2025-08-20 16:53:05", "rationale": "Because this company looks promising"}], "portfolio_value_time_series": [["2025-08-20 16:53:05", 9999.20332], ["2025-08-20 16:53:05", 9999.20332], ["2025-08-20 16:53:18", 9999.20332]], "total_portfolio_value": 9999.20332, "total_profit_loss": -0.7966799999994691}
